In [ ]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2

import cv2
import numpy as np
import rospy, rosbag, tf
import os, csv, time, argparse, math
from matplotlib import pyplot as plt

from nav_msgs.msg import Odometry
from geometry_msgs.msg import Twist
from gazebo_msgs.msg import ModelStates
from sensor_msgs.msg import Image, CameraInfo
from cv_bridge import CvBridge, CvBridgeError
from tf.transformations import euler_from_quaternion, quaternion_from_euler

from src.VBOATS import VBOATS

In [ ]:
class image_converter:
    def __init__(self):
        rospy.init_node('vboat_pipeline')
        self.bridge = CvBridge()
        self.image_pub = rospy.Publisher("/obstacles",Image,queue_size=1000)
        self.image_sub = rospy.Subscriber("/ugv1/d415/depth/image_raw",Image,self.callback)
        
        self.vboat = VBOATS()
        self.vboat.dead_x = 0
        self.vboat.dead_y = 6
        self.r = rospy.Rate(40)
        self.img = []
        self.obs_disp = []
        self.flag_initialization = False
        self.nSamples = 100
        self.dSum = 0
        self.dmaxAvg = 0
        self.count = 0
        self.disp_obs = None
        
    def callback(self,data):
        try:
            cv_image = self.bridge.imgmsg_to_cv2(data, "16UC1")
            cv_image = np.float32(cv_image)
        except CvBridgeError as e: print(e)
                
        if(not self.flag_initialization):
            tmp = cv_image/65535
            depth = np.uint8(tmp*255)
            self.img = np.copy(depth)
            self.vboat.pipeline(depth, threshU1=11,threshU2=20, threshV2=70)
            display_obstacles = cv2.cvtColor(self.vboat.img, cv2.COLOR_GRAY2BGR)

            for ob in self.vboat.obstacles:
                cv2.rectangle(display_obstacles,ob[0],ob[1],(150,0,0),1)

            self.disp_obs = np.copy(display_obstacles)
            try:
                self.obs_disp = np.copy(display_obstacles)
                self.image_pub.publish(self.bridge.cv2_to_imgmsg(display_obstacles, "bgr8"))
            except CvBridgeError as e:
                print(e)
            
    def start(self):
        while not rospy.is_shutdown():
            self.r.sleep()

In [ ]:
io = image_converter()

In [ ]:
plt.figure(1)
plt.imshow(io.vboat.umap_raw)
plt.show()

plt.figure(2)
plt.imshow(io.vboat.umap_processed)
plt.show()

plt.figure(3)
plt.imshow(io.vboat.vmap_raw)
plt.show()

In [ ]:
# D415 Parameters returned from real camera
rotation = [1, 0.000451149, 0.000541019, -0.000453043, 0.999994, 0.0035053, -0.000539434, -0.00350554, 0.999994]
translation = [0.0150125, 2.66028e-05, -0.000210271]
intrinsics = {
    "width": 640, "height": 480, 
    "ppx": 320.551, "ppy": 232.202, 
    "fx": 626.464, "fy": 626.464, 
    "model": "Brown Conrady", 
    "coeffs": [0, 0, 0, 0, 0]
}

F = np.array([
    [intrinsics['fx'], 0, intrinsics['ppx']],
    [0, intrinsics['fy'], intrinsics['ppy']],
    [0,0,1]
])
R = np.array(rotation).reshape((3, 3))
Rinv = np.linalg.inv(R)
T = np.array(translation).reshape((3, 1))
T2 = np.array([intrinsics['ppx']*0.001, intrinsics['ppy']*0.001, 0, 0]).T
print T.T
print T2.T

In [ ]:
def get_camera_pose(cam_frame = '/ugv1/d415_camera_depth_optical_frame',base_frame = '/ugv1/odom'):
    listener = tf.TransformListener()
    listener.waitForTransform(base_frame,cam_frame, rospy.Time(0), rospy.Duration(8.0))
    (trans,rot) = listener.lookupTransform(base_frame,cam_frame, rospy.Time(0))
    roll,pitch,yaw = tf.transformations.euler_from_quaternion(rot)

    pose = np.array(trans+[np.rad2deg(yaw)])
    Tmat = np.array(trans).T
    Rmat = tf.transformations.euler_matrix(roll,pitch,yaw,axes='sxyz')
    return pose, Rmat, Tmat

def get_distance(umap, xs, ds, ys, focal=[462.138,462.138], baseline=0.055, dscale=0.001, pp=[320.551,232.202], method=0, use_principle_point=False):
        nonzero = umap.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])

        good_inds = ((nonzeroy >= ds[0]) & (nonzeroy < ds[1]) &
                     (nonzerox >= xs[0]) &  (nonzerox < xs[1])).nonzero()[0]

        xmean = np.int(np.mean(nonzerox[good_inds]))
        ymean = np.mean(ys)
        dmean = np.mean(nonzeroy[good_inds])
        
        if(method == 1): 
            dist = (focal[0]*baseline)/dmean
            z = dist*dscale
        else:
#             z = np.min(ds)*(65535/255)*dscale
            z = dmean*(65535/255)*dscale

        if use_principle_point:
            px = pp[0]
            py = pp[1]
        else:
            px = 0
            py = 0
            
        x = (xmean - px)/focal[0]
        y = (ymean - py)/focal[1]
        
        x = x * z
        y = y * z
#         print("Distance, X, Y: %.3f, %.3f, %.3f" % (z, x,y))
        return z, x,y, dmean

In [ ]:
test_terra = False
umap = io.vboat.umap_raw
xs = io.vboat.xBounds
ds = np.array(io.vboat.dbounds)
obs = io.vboat.obstacles

ppx = 320.551; ppy = 232.202; fx = 626.464; fy = 626.464; b = 0.055
pp = [ppx, ppy];   focal = [fx,fy]
dGain = (65535/255)

if(test_terra):
    obid = 2
    i = 1
    # Obstacles terra world
    X0 = np.array([
        [4.0, -1.0, 0.227],
        [3.0, 0, 0.177],
        [2.0, -2.0, 0.344]
    ])
else:
    obid = 0
    i = 0
    # Obstacles offset world
#     X0 = np.array([
#         [11.59, -3.6, 0.21336],
#         [8.576193, -1.165787, 0.21336],
#         [5.694473, 2.2, 0.21336]
#     ])
    X0 = np.array([ [5.0, -3.0, 0.21336] ])


pose,RotM,Tmat = get_camera_pose()
RotM = RotM[:3,:3]
Xk = pose[:2]; yaw = pose[-1]; ang = np.deg2rad(yaw)
x1 = X0[obid,0];     y1 = X0[obid,1]
dx = x1 - Xk[0];     dy = y1 - Xk[1]
true_dist = np.sqrt(dx*dx + dy*dy)-X0[obid,2]

print "Obstacle Disparities: ", ds

d = np.mean(ds[i,0])#-1
z2 = d * dGain * 0.001

# Method 2: --------
disparities = ds[i]
disparities[0] = disparities[0] - 1
us = [obs[i][0][0], obs[i][1][0]]
vs = [obs[i][0][1], obs[i][1][1]]
z,ux,uy,_ = get_distance(umap,us,disparities,vs, focal=focal,pp=pp,use_principle_point=True)


theta = math.atan((ux/z))
heading = math.atan(dy/dx)
print(yaw,np.rad2deg(heading),np.rad2deg(theta))

# print("True Dist, Est. Dist (Method #1), (Method #2): %.3f, %.3f, %.3f" % (true_dist, z, z2) )

pxl = np.array([ [ux],[uy],[z] ])
RotMinv = np.linalg.inv(RotM.T)
print(', '.join(map(str, np.around(Tmat.T,3))))
Tmat = Tmat.reshape((3, 1))*-1
pos = np.dot(RotMinv, pxl)
tmp1 = pos - Tmat
# tmp1[0] = tmp1[0] + Xk[0]

print "Camera Location: ", Xk#, yaw
# print "Obstacle Estimated Location: ", tmp1.T[0]
# print "Obstacle True Location: ", X0[obid,:2]

strs = []
strs.append(', '.join(map(str, np.around(pxl.T[0],3))))
strs.append(', '.join(map(str, np.around(pos.T[0],3))))
strs.append(', '.join(map(str, np.around(Tmat.T[0],3))))
strs.append(', '.join(map(str, np.around(tmp1.T[0][:2],3))))
strs.append(', '.join(map(str, np.around(X0[obid,:2],3))))
# strs.append(', '.join(map(str, np.around(tmp1.T[0],3))))

print(
"""
Detected Obstacle Stats:
========================

    * Distances (True, Est.)    : %.3f, %.3f
    * Pixel (X,Y,Z)             : %s
    * Rotated Position (X,Y,Z)  : %s
    * Translation (X,Y,Z)       : %s
    
    * Estimated Position (X,Y,Z): %s
    * True Position (X,Y,Z)     : %s
""" % (true_dist,z,strs[0],strs[1],strs[2],strs[3],strs[4]) )

In [ ]:
strs = []
strs.append(', '.join(map(str, np.around(pxl.T[0],3))))
strs.append(', '.join(map(str, np.around(pos.T[0],3))))
strs.append(', '.join(map(str, np.around(tmp1.T[0],3))))
strs.append()
# strs.append(', '.join(map(str, np.around(tmp1.T[0],3))))

print("Test: %s"%test)

In [ ]:
pxl = np.array([ [ux],[uy],[z] ])
RotMinv = np.linalg.inv(RotM.T)
# Tmat = Tmat.reshape((3, 1))*-1

pos = np.dot(RotMinv, pxl)

tmp1 = pos - Tmat
tmp1[0] = tmp1[0] + Xk[0]

print "Pixel (X,Y,Z): ", pxl.T[0]
print "Position (X,Y,Z): ", pos.T[0]
print Tmat.T
print "Temp (X,Y,Z): ", tmp1.T

print "Camera Location: ", Xk, yaw
print "Obstacle Estimated Location: ", tmp1.T
print "Obstacle True Location: ", X0[obid,:2]

In [ ]:
xoff = np.tan(ang)*z
yoff = z / np.cos(ang)
zoff = (fx*0.001)/np.tan(ang)

offs = [xoff, yoff, zoff]
location = [yoff, np.asscalar(xoff-pos[0])]
# location = [yoff, np.asscalar(xoff+pos[0][0])]

print "Offsets: ", offs
print "Obstacle Estimated Location: ", location

In [ ]:
tpp = [0,1]
t,p = tpp 
print(t,p)

In [ ]:
bot = [0.444806, -0.559817]
obs = [2.000000, -2]
dx = bot[0] - obs[0]
dy = bot[1] - obs[1]
dist = np.sqrt(dx*dx + dy*dy)
print(dist)

In [ ]:
win = io.vboat.obstacles_umap[0]
print(win)
crop_img = io.vboat.umap_raw[win[0][1]:win[1][1], win[0][0]:win[1][0]]

tmpMean = np.mean(crop_img)
print(tmpMean)

plt.figure(3)
plt.imshow(crop_img)
plt.show()

In [ ]:
# cam_frame = '/ugv/d415_camera_depth_optical_frame'
cam_frame = '/ugv/d415_camera_depth_optical_frame'
base_frame = '/ugv/odom'
listener = tf.TransformListener()
listener.waitForTransform(base_frame,cam_frame, rospy.Time(0), rospy.Duration(8.0))
(trans,rot) = listener.lookupTransform(base_frame,cam_frame, rospy.Time(0))
roll,pitch,yaw = tf.transformations.euler_from_quaternion(rot)

print
print np.rad2deg([roll,pitch,yaw])
print
print(tf.transformations.quaternion_matrix(rot))
print
rotot = tf.transformations.euler_matrix(roll,pitch,yaw).T
print(rotot)
print 
R11 = np.cos(pitch)*np.cos(yaw)
R12 = np.cos(pitch)*np.sin(yaw)
R13 = -np.sin(pitch)
R1 = [R11, R12, R13]

R21 = -(np.cos(roll)*np.sin(yaw)) + (np.sin(roll)*np.sin(pitch)*np.cos(yaw))
R22 = (np.cos(roll)*np.cos(yaw)) + np.sin(roll)*np.sin(pitch)*np.sin(yaw)
R23 = np.sin(roll)*np.cos(pitch)
R2 = [R21,R22,R23]

R31 = (np.sin(roll)*np.sin(yaw))+(np.cos(roll)*np.sin(pitch)*np.cos(yaw))
R32 = -(np.sin(roll)*np.cos(yaw)) + (np.cos(roll)*np.sin(pitch)*np.sin(yaw))
R33 = np.cos(roll)*np.cos(pitch)
R3 = [R31,R32,R33]

testRot = np.array([R1,R2,R3])
print(testRot)
print(testRot - rotot[:3,:3])

In [ ]:
pxl = np.array([ [ux],[uy],[z] ])
RotMinvT = np.linalg.inv(testRot)
# Tmat = Tmat.reshape((3, 1))*-1

poss = np.dot(RotMinvT, pxl)

tmp1s = poss - Tmat
# tmp1s[0] = tmp1s[0] + Xk[0]

print "Pixel (X,Y,Z): ", pxl.T[0]
print "Position (X,Y,Z): ", poss.T[0]
print Tmat.T
print "Temp (X,Y,Z): ", tmp1s.T

print "Camera Location: ", Xk
print "Obstacle Estimated Location: ", tmp1s.T
print "Obstacle True Location: ", X0[obid,:2]

In [ ]:
tmp = np.dot(mat,np.vstack([pos,1]))
print tmp
print
print pos
print
print xoff,yoff,zoff

In [ ]:
(t,r) = listener.lookupTransform("ugv/base_link",cam_frame, rospy.Time(0))
print(t)

In [ ]:
math.atan((0.5*640)/626.464)/0.5